## DT8060
## Raffaello Baluyot

# Lab 2 - Interpretable Models

In this lab, we will venture through the simpler, interpretable, models Linear Regression and Decision Trees.
In part 1, linear regression will be applied to a housing dataset with houses for sale in California. The weights and the effect of the weights will be investigated to see how each feature affects the model and how using subset of features makes the models become more interpretable but at what cost of the predictive performance.

In part 2, decision trees will be created to model a car safety dataset, and we will investigate their performance and how interpretable they are, with the use of graphical aids such as **graphviz** and **dtreeviz**.

In the 3rd and final part, a Naive Bayes classifier is given to on a spam dataset. A Naive Bayes classifier operates naively by seeing each feature as independent and use their probabilities to determine its class. You are to investigate the model that is produced to try and explain what dictates the classification.

# Linear Regression
In this part of the lab, we train a linear regression model on a dataset containing housing prices from California. The target value is the price of the house.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from sklearn.datasets import fetch_california_housing


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Load datasets

In [ ]:
data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'])

## Train the model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)

## Inspect the model
Presenting the weights of each attribute in the model gives us an indication of the importance of the feature and its values.

In [ ]:
df = pd.DataFrame()
df['Weight'] = np.append(lr.intercept_,lr.coef_)
df.index=['Intercept'] + data['feature_names']

df

Just to show the distribution of the data, we provide a boxplot below so you see the scale for each of the features.

In [ ]:
x_df = pd.DataFrame(X_train, columns=data['feature_names'])
sns.boxplot(data=x_df, orient='h').set(ylabel='Features', xlabel='Weights')

### Feature Effect

To visualize how each feature affects the prediction from the linear regression model we can look at the weights and its variance, but if we have a large difference in feature values it could be hard to interpret.

If we instead calculate the effect of each feature and visualize that, we get a better indication of how the features affect the predictions.

The effects of a feature on a single data instance is calculated as $$\text{effect}_{j}^{(i)}=w_{j}x_{j}^{(i)}$$
where $w_{j}$ is the weight of feature $j$, and $x_{j}^{(i)}$ is the value of feature $j$ in data instance $i$.

We visualize the effect using boxplots, which show the median values (the line inside the box), the 1st to 3rd quartiles (the actual boxes), and the outliers (the dots).

In [ ]:
plt.rcParams['figure.figsize'] = [16, 9]
sns.set_theme(rc={"figure.dpi": 96})

weights = lr.coef_
effects = np.multiply(X_train, weights)

df = pd.DataFrame(effects, columns=data['feature_names'])
sns.boxplot(data=df, orient='h').set(title='Feature Effect', ylabel='Features', xlabel='Effect')

We can also show how the feature effects of single instances. This is a good indicator of the data, especially when it is plotted in combination with the effects of the entire model.

In [ ]:
instance = np.multiply(X_train[10], weights)
fix, ax = plt.subplots(1,1)
plot = sns.boxplot(data=df, orient='h')
plot = plot.set_title('Feature Effect')
y_axis = [y for y in range(8)]

ax.scatter(instance, y_axis, marker='x', color='r', s=70, zorder=10)

What can you say about the results from the above plots? What does it mean if we compare the effect from the latitude and logitude features? What is their effect on the model?

**Answer the question above here.**

In general, the **latitude** and **longitude** features have the largest impact on then model result. The higher **latitude** means lower price (northen areas are cheaper) and the higher **longitude** means means more expensive (western areas are more expensive). Given that the model also gives significantly higher weights to these two features means that location is one of the main factors when it comes to house pricing.


### Lasso
The more features you have, the more you affect the complexity of the linear regression model. It is an interpretable model when we have a modest number of features. When we increase the amount of features the interpretability deteriorates. But, how do we decide which features to discard? 

Using a domain expert is often a good choice. Likewise, identifying which features that have the highest correlation with the target could be a way to go.

An automated approach called Lasso investigates which features provide the most for the model.

In this example below, we investigate which two features provides the better model using Lasso.

In [ ]:
from sklearn.feature_selection import SelectFromModel

# threshold=-np.inf forces Lasso to pick the number of features specified
sel_ = SelectFromModel(LinearRegression(), threshold=-np.inf, max_features=2)
sel_.fit(X_train, y_train)
sel_.get_support()

X_train_selected = sel_.transform(X_train)
X_test_selected = sel_.transform(X_test)
lr_selected = LinearRegression()
lr_selected.fit(X_train_selected, y_train)
y_pred = lr_selected.predict(X_train_selected)

Identify which models are better performing, by adapting the code from above, for each number of features ranging from 1 to 8 and store them for later use.

In [ ]:
import dataclasses as dc

from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error


@dc.dataclass
class TrainResult:
    model: LinearRegression
    selector: SelectFromModel

    def predict(self, X):
        X_selected = self.selector.transform(X)
        return self.model.predict(X_selected)
    
    def eval(self, X, y):
        pred = self.predict(X)
        return mean_squared_error(y, pred)
    
    def plot_feature_effects(self, X):
        X_selected = self.selector.transform(X)
        effects = np.multiply(X_selected, self.model.coef_)
        idx_selected = self.selector.transform(np.arange(self.selector.n_features_in_)[None, :])[0]
        cols_selected = [data["feature_names"][i] for i in idx_selected]
        sns.boxplot(data=pd.DataFrame(effects, columns=cols_selected), orient='h')
        plt.title('Feature Effect')
        plt.show()

def train_by_n_features(n_features):
    sel_ = SelectFromModel(LinearRegression(), threshold=-np.inf, max_features=n_features)
    sel_.fit(X_train, y_train)
    sel_.get_support()

    X_train_selected = sel_.transform(X_train)
    lr_selected = LinearRegression()
    lr_selected.fit(X_train_selected, y_train)

    return TrainResult(lr_selected, sel_)

In [ ]:
trained_models = {i: train_by_n_features(i) for i in range(1, 8+1)}

Evaluate the predictive performance of the chosen linear models on the data to see how they compare with each other.

Produce a plot which shows the root mean squared error for each of the chosen models, both on the training and testing data.

In [ ]:
train_performance = {idx: res.eval(X_train, y_train) for idx, res in trained_models.items()}
test_performance = {idx: res.eval(X_test, y_test) for idx, res in trained_models.items()}

In [ ]:
def plot_performance(performance, name):
    plt.plot(pd.Series(performance))
    plt.title(f"{name}: Mean Squared Error vs N Features")
    plt.xlabel("N Features")
    plt.ylabel("Mean Squared Error")
    plt.show()

plot_performance(train_performance, "Train")
plot_performance(train_performance, "Test")

Look at the feature effects on the different models and show how they differ between themselves.

In [ ]:
for n_features, trained_model in trained_models.items():  
    print(f"{n_features} Features")
    trained_model.plot_feature_effects(X_train)

Depending on the model and the number of features, the features themselves have different effect on the models. How did the features effect the models? Was there one feature that consistently had a significant effect or was there a combination that showed better predictive performances?

**answer above question here**

Similar to using the 8 features, the location has presented the best effect on the models. Though it's interesting that the **longitude** alone does not produce a very high effect. Only when it is combined with the **latitude** that you get a very good result.

Another observation here is the importance of data normalization. The weights of the linear regression model is impacted by the range of the values of the inputs. This is why despite having pretty good effect on the 8 feature model, the first two features selected by the `SelectFromModel` module were not location related. **AveBedrms** and **MedInc** are two of the lowest when it comes to the data magnitude, and provided very high coefficients despite not having a very good effect.

# Decision Trees
In this part we train a decision tree on a dataset that contains information about cars and their safety class.

## Load datasets

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/'
data_url = url + 'car.data'
columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv(data_url, header=None)
df.columns = columns
ct = ColumnTransformer([('categorical', OrdinalEncoder(), df.columns)])
ct.fit(df)
df = ct.transform(df)
df = pd.DataFrame(df, columns=columns)

X = df.drop(['class'], axis=1)
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

# dtreeviz does not like if the target class is not of int type, casting the targets to ints.
y_train = y_train.astype(int)
y_test = y_test.astype(int)

Here we train the tree with a maximum depth of 4. Play around with the depth to see how the interpretability of the tree changes as the depth increases.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

tree_depth=7
gini_tree = DecisionTreeClassifier(criterion='gini', max_depth=tree_depth)
gini_tree.fit(X_train.values, y_train.values)
print(classification_report(y_test.values, gini_tree.predict(X_test.values)))

To show how the tree is built and how data is divided at each leaf, we can use both graphviz and dtreeviz to visualize the produced rules. It is an easier and more pleasant view of the tree model compared to written rules.

In [ ]:
from sklearn import tree
import graphviz

g = tree.export_graphviz(gini_tree, 
                         feature_names=X_train.columns.tolist(),
                         filled=True)
graphviz.Source(g)

In [ ]:
import dtreeviz

viz = dtreeviz.model(gini_tree, X_train, y_train, 
                     target_name='class', 
                     feature_names=X_train.columns.tolist(), 
                     class_names=df['class'].unique().tolist())
viz.view()


As we increase the heigt of the tree it is commonly so that the predictive performance increases, but as you saw it becomes quite messy quite quickly when you increase the height. 

Taking both interpretability into account and the predictive performance of the model, at which tree height did you find to strike a good balance?

**answer above question here**

Performance is the biggest bottle neck, since in tree depth less than 7, the model cannot predict some of the classes on the test set. Assuming that having a balance on the labels is very much important, this is the minimum acceptable performance. However, 7 is already very complex from a explainability perspective. I would say this is the best balance since this is the lowest depth with acceptable performance.

# Naive Bayes Classifier
In this part of the lab we will produce a Naive Bayes model on a spam dataset. The model should then be used to determine if the message is regarded as spam (malicious) or ham (benevolent).

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Dataset downloaded from https://archive.ics.uci.edu/ml/datasets/sms+spam+collection
spam_df = pd.read_csv('../data/SMSSpamCollection.csv')
spam_df['spam'].value_counts()

In [ ]:
cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(spam_df['text'])
X_train, X_test, y_train, y_test = train_test_split(X, spam_df['spam'], test_size=0.3, random_state=42)


In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb.score(X_test, y_test)

Identify 10 words that you want to investigate. The words are available in the list below when you run the cell.

In [ ]:
# Just select which ever 10 words

import itertools
target_words = list(itertools.islice(cv.vocabulary_.keys(), 10))
target_words

Gather the individual probabilities for the 10 words you have chosen and explain how they impact the model. In the cell below we identify the mapping of the word hello and gather its probability in the form \[ham, spam\].

In [ ]:
a = cv.transform(target_words)
nb.predict_proba(a)

**Explain the words impact on the model based on the probabilities above**

Gather the 10 words that most indicate it to be a spam email and the 10 words that mostly indicate for not being spam. Do they make sense?

**Answer**

The words used for spam indicator makes sense given that most spam emails always provide some sort of compensation for you to act on something. However, the non words are mostly just gibberish and are probably not considered spam just by the rarity that these words appear on emails. They probably don't appear as well on the example spams.

In [ ]:
vocabs_proba = nb.predict_proba(cv.transform(list(cv.vocabulary_.keys())))
vocabs = pd.DataFrame({
    "words": list(cv.vocabulary_.keys()),
    "ham": vocabs_proba[:, 0],
    "spam": vocabs_proba[:, 1],
})

In [ ]:
vocabs.sort_values("ham", ascending=False).head(10)

In [ ]:
vocabs.sort_values("spam", ascending=False).head(10)

# Final reflection
What have you learned throughout this lab? This includes but is not limited to the example points below.

- Are the models that we presented interpretable under any condition? 

- Would, for instance, the predictions from decision tree with 1000 levels be easy to decipher? 

- Would a linear regression model with 123456 attributes be interpretable?

Reason and reflect about the interpretable models.

**Answer**

The models are interpretable within a certain level of complexity. Once these models become too complex or too large, they also lose their ability to be comprehensible. That being said, these models still provide some way to give us insights even if they are too complex. For example, linear regression still provide the weights as a summary of interpretability while decision trees and forest approach can provide their weight importances. Similarly bayesian models can be sampled to check if their predictions makes sense.

In summary, interpretable models can be difficult to understand once their complexity grows, but they still provide some avenues to gain insights on their decision making process.